In [1]:
!pip install ultralytics==8.0.106

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 7.9 MB/s eta 0:00:00:00:010:01


In [2]:
import ultralytics

In [3]:
import torch

In [4]:
pip install deep_sort_realtime 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
from deep_sort_realtime.deepsort_tracker import DeepSort

In [6]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Initialize YOLO model
model = YOLO("yolov8n.pt")  # YOLOv8n is a lightweight model

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=40, n_init=3)

# Define video path and output path
video_path = '/kaggle/input/data-set/Play Therapy Session working on Feelings with Candy Land Game (online-video-cutter.com).mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_path = 'outputvideo.mp4'

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB as YOLO expects
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform detection using YOLO
    results = model(rgb_frame, device='cpu')

    detections = []
    for result in results:
        boxes = result.boxes  # Get the bounding boxes
        cls = boxes.cls.tolist()  # Get the classes
        xyxy = boxes.xyxy.cpu().numpy()  # Get the bounding boxes in (x1, y1, x2, y2) format
        confs = boxes.conf.cpu().numpy()  # Get the confidence scores

        for i in range(len(boxes)):
            # Only track the 'person' class (class ID = 0)
            if int(cls[i]) == 0:
                x1, y1, x2, y2 = map(int, xyxy[i])
                conf = confs[i]
                
                # Filter out weak detections
                if conf > 0.6:  # Confidence threshold
                    detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    # Update tracker with new detections
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw bounding boxes and IDs on the frame
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()  # Get the bounding box coordinates in left, top, right, bottom format
        x1, y1, x2, y2 = map(int, ltrb)

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw the unique ID
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

    # Write the frame to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()

print(f"Processed video saved as {output_path}")
cv2.destroyAllWindows()




/opt/conda/lib/python3.10/site-packages/ultralytics/nn/tasks.py:381: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


Processed video saved as outputvideo.mp4


error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
